In [40]:
from bs4 import BeautifulSoup
import urllib.request

In [41]:
from IPython.display import display, clear_output

In [42]:
import urllib.parse

In [43]:
import pandas as pd

In [44]:
orang = list(open('rp.csv'))

In [45]:
orang = [x.strip() for x in orang]

In [46]:
orang

['http://dbpedia.org/resource/Elizabeth_II',
 'http://dbpedia.org/resource/George_VI',
 'http://dbpedia.org/resource/Queen_Elizabeth_The_Queen_Mother',
 'http://dbpedia.org/resource/Prince_Philip,_Duke_of_Edinburgh',
 'http://dbpedia.org/resource/Anne,_Princess_Royal',
 'http://dbpedia.org/resource/Charles,_Prince_of_Wales',
 'http://dbpedia.org/resource/Prince_Edward,_Earl_of_Wessex',
 'http://dbpedia.org/resource/Prince_Andrew,_Duke_of_York',
 'http://dbpedia.org/resource/Mark_Phillips',
 'http://dbpedia.org/resource/Timothy_Laurence',
 'http://dbpedia.org/resource/Zara_Phillips',
 'http://dbpedia.org/resource/Peter_Phillips',
 'http://dbpedia.org/resource/Diana,_Princess_of_Wales',
 'http://dbpedia.org/resource/Camilla,_Duchess_of_Cornwall',
 'http://dbpedia.org/resource/Prince_William,_Duke_of_Cambridge',
 'http://dbpedia.org/resource/Prince_Harry',
 'http://dbpedia.org/resource/Prince_George_of_Cambridge',
 'http://dbpedia.org/resource/Princess_Charlotte_of_Cambridge',
 'http://db

In [47]:
def get_dbo_parent(dbr, match, herit):
    # clear_output(wait=True)
    # display(len(match), herit, dbr)
    if herit >= 20:
        return match
    if dbr in [m[0] for m in match]:
        return match
    url = urllib.parse.urlsplit(dbr)
    url = list(url)
    url[2] = urllib.parse.quote(url[2])
    url = urllib.parse.urlunsplit(url)
    page = urllib.request.urlopen(url)
    soup = BeautifulSoup(page, 'html.parser')
    docs = soup.find_all('a', attrs={'rel': 'dbo:parent'})
    link = []
    for num,x in enumerate(docs):
        match = get_dbo_parent(x.get('href'), match, herit+1)
        link.append(x.get('href'))
    match.append([dbr, link])
    return match

In [10]:
# n = []
result = []
result = get_dbo_parent('http://dbpedia.org/page/Prince_Harry', result, 0)

In [11]:
len(result)

899

In [12]:
import pickle
with open('harry2.pkl','wb') as f:
    pickle.dump(result,f)

In [48]:
result = []
for url in orang:
    result = get_dbo_parent(url, result, 0)

In [49]:
len(result)

1315

In [50]:
queries_match = []
queries_match_new = []
queries_create = []
p = 0
for x in result:
    for y in x[1]:
        if y in orang:
            query = "MATCH (a:Person),(b:Person) WHERE  a.`dbr` = '" +x[0]+ "' AND b.`dbr` = '" +y+ "' CREATE (a)-[r:`dbo:parent` {URI :'http://dbpedia.org/ontology/parent'}]->(b) RETURN type(r)"
            queries_match.append(query)
        else:
            print(y)
            p += 1
            p_str = 'p' + str(p)
            query = "CREATE("+p_str+":Person {`dbr`: '" +y+ "'})"
            queries_create.append(query)
            query = "MATCH (a:Person),(b:Person) WHERE  a.`dbr` = '" +x[0]+ "' AND b.`dbr` = '" +y+ "' CREATE (a)-[r:`dbo:parent` {URI :'http://dbpedia.org/ontology/parent'}]->(b) RETURN type(r)"
            queries_match_new.append(query)

http://dbpedia.org/resource/Joanna_of_Hainaut
http://dbpedia.org/resource/William_V,_Duke_of_Jülich
http://dbpedia.org/resource/Gerhard_VI_of_Jülich,_Count_of_Berg_and_Ravensberg
http://dbpedia.org/resource/Margaret_of_Ravensberg
http://dbpedia.org/resource/William_VII_of_Jülich,_1st_Duke_of_Berg
http://dbpedia.org/resource/William_VIII_of_Jülich,_Count_of_Ravensberg
http://dbpedia.org/resource/John_III,_Count_of_Holstein-Plön
http://dbpedia.org/resource/Eric_I,_Duke_of_Saxe-Lauenburg
http://dbpedia.org/resource/Elisabeth_of_Pomerania_(1291–1349)
http://dbpedia.org/resource/Agnes_of_Holstein
http://dbpedia.org/resource/Eric_II_of_Saxe-Lauenburg
http://dbpedia.org/resource/Eric_IV_of_Saxe-Lauenburg
http://dbpedia.org/resource/Bernard_II,_Duke_of_Saxe-Lauenburg
http://dbpedia.org/resource/Gerhard_VII,_Duke_of_Jülich-Berg
http://dbpedia.org/resource/Sophie_of_Saxe-Lauenburg
http://dbpedia.org/resource/Mastino_II_della_Scala
http://dbpedia.org/resource/Doria_(family)
http://dbpedia.org/res

http://dbpedia.org/resource/Elisabeth_Sophie_of_Saxe-Altenburg
http://dbpedia.org/resource/Ernest_I,_Duke_of_Saxe-Gotha
http://dbpedia.org/resource/Josias_I,_Count_of_Waldeck
http://dbpedia.org/resource/Wolrad_IV,_Count_of_Waldeck
http://dbpedia.org/resource/Prince_Georg_Friedrich_of_Waldeck
http://dbpedia.org/resource/Ernest,_Duke_of_Saxe-Hildburghausen
http://dbpedia.org/resource/Countess_Sophie_Henriette_of_Waldeck
http://dbpedia.org/resource/Ernst_Frederick_I,_Duke_of_Saxe-Hildburghausen
http://dbpedia.org/resource/Countess_Sophia_Albertine_of_Erbach-Erbach
http://dbpedia.org/resource/Adolf_Frederick_I,_Duke_of_Mecklenburg-Schwerin
http://dbpedia.org/resource/Adolf_Frederick_II,_Duke_of_Mecklenburg-Strelitz
http://dbpedia.org/resource/Princess_Elizabeth_Albertine_of_Saxe-Hildburghausen
http://dbpedia.org/resource/Duke_Charles_Louis_Frederick_of_Mecklenburg,_Prince_of_Mirow
http://dbpedia.org/resource/George_III_of_the_United_Kingdom
http://dbpedia.org/resource/Charlotte_of_Mecklenb

In [51]:
print("MATCH:", len(queries_match), "CREATE:", len(queries_create), "MATCH NEW:", len(queries_match_new))

MATCH: 90 CREATE: 2013 MATCH NEW: 2013


In [52]:
print(orang)

['http://dbpedia.org/resource/Elizabeth_II', 'http://dbpedia.org/resource/George_VI', 'http://dbpedia.org/resource/Queen_Elizabeth_The_Queen_Mother', 'http://dbpedia.org/resource/Prince_Philip,_Duke_of_Edinburgh', 'http://dbpedia.org/resource/Anne,_Princess_Royal', 'http://dbpedia.org/resource/Charles,_Prince_of_Wales', 'http://dbpedia.org/resource/Prince_Edward,_Earl_of_Wessex', 'http://dbpedia.org/resource/Prince_Andrew,_Duke_of_York', 'http://dbpedia.org/resource/Mark_Phillips', 'http://dbpedia.org/resource/Timothy_Laurence', 'http://dbpedia.org/resource/Zara_Phillips', 'http://dbpedia.org/resource/Peter_Phillips', 'http://dbpedia.org/resource/Diana,_Princess_of_Wales', 'http://dbpedia.org/resource/Camilla,_Duchess_of_Cornwall', 'http://dbpedia.org/resource/Prince_William,_Duke_of_Cambridge', 'http://dbpedia.org/resource/Prince_Harry', 'http://dbpedia.org/resource/Prince_George_of_Cambridge', 'http://dbpedia.org/resource/Princess_Charlotte_of_Cambridge', 'http://dbpedia.org/resource

In [53]:
df = pd.DataFrame(queries_create)

In [54]:
df.to_csv('query_create.csv', header=False, index=False, sep='!')

In [55]:
df = pd.DataFrame(queries_match)

In [56]:
df.to_csv('query_match.csv', header=False, index=False, sep='!')

In [57]:
df = pd.DataFrame(queries_match_new)

In [58]:
df.to_csv('query_match_new.csv', header=False, index=False, sep='!')